Calculate ensemble of sea level projections

In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append('../src')

from alldata import AllData
from ensemble import EnsembleMember,FullEnsemble,FullEnsemble2

np.set_printoptions(precision=2)
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

Read data

In [ ]:
option = 'calibrated'
#option = 'fixed'

In [ ]:
ad = AllData()
ad.year0 = 1871
ad.usefanom = True
ad.gather(update=False)
#Select subsection of models for rapid calculation
ad.ds = ad.ds.drop_sel(ism='PISM_DMI')
#ad.ds = ad.ds.isel(ssp=slice(0,2))
#ad.ds = ad.ds.isel(ism=slice(0,2),esm=slice(0,2))

if option=='calibrated':
    fe = FullEnsemble(ad)
elif option=='fixed':
    fe = FullEnsemble2(ad)
fe.nonlin = 'cutoff'
fe.lump = True
fe.gather(force_update=False)

In [ ]:
#Rignot data from Dewi

def ant_rignot19():
    '''Use data of mass balance from table 2 of Rignot et al. 2019. 
    Fit a second order polynomial through these data that covers 1979 to 2017. 
    Extend to 1950 assuming that Antarctica did not loose mass before 1979.'''
    ye = 2019 # Last year plus 1
    dM_79_89 = 40    # Gt/y
    dM_89_99 = 49.6
    dM_99_09 = 165.8 
    dM_09_17 = 251.9
    #Fit a second order polynomial to the data
    xy = np.array([1984, 1994, 2004, 2013])
    dM = [dM_79_89, dM_89_99, dM_99_09, dM_09_17]
    dM2f = np.polyfit(xy - xy[0], dM, 2)
    xy2 = np.arange(1979,ye)
    dM2 = dM2f[0] * (xy2 - xy[0])**2 + dM2f[1] * (xy2 - xy[0]) + dM2f[2]
    slr_rig = dM2.cumsum() / 3600 # Convert from Gt to cm
    slr_rig_df = pd.DataFrame(data = dict(time= xy2, Antarctica = slr_rig))
    slr_rig_df = slr_rig_df.set_index('time')
    slr_rig_df = slr_rig_df.reindex(np.arange(1950,ye)).fillna(0)
    return slr_rig_df

slr_rig_df = ant_rignot19()

Figure settings

In [ ]:
plt.style.use('../src/style_paper')
mpl.rc('figure.subplot',left=.1,bottom=.15,wspace=.4,top=.9)

alpha = .2 #Transparency of uncertainty range
perc = [17,83] #Percentiles of uncertainty range

Make and save figure

In [ ]:
fig,ax = plt.subplots(1,2,sharex=True,figsize=(7,3.5))


#Total sea level rise
dax = ax[0]

#Plot Rignot
dax.plot(slr_rig_df[29:],c='tab:olive',lw=4,zorder=10,label='Observations')

#Plot no feedback
var = 100*(fe.slr[0,1,:,:,:]-fe.slr[0,1,:,:,1979-fe.year0])
dax.plot(fe.time,np.nanmedian(var,axis=(0,1)),c='.5',lw=2,label='No feedback')
dax.fill_between(fe.time,np.nanpercentile(var,perc[0],axis=(0,1)),np.nanpercentile(var,perc[1],axis=(0,1)),color='.5',alpha=alpha)#,label=perc)

#Plot with feedback
var = 100*(fe.slr[1,1,:,:,:]-fe.slr[1,1,:,:,1979-fe.year0])
dax.plot(fe.time,np.nanmedian(var,axis=(0,1)),c='tab:green',lw=2,label='With feedback')
dax.fill_between(fe.time,np.nanpercentile(var,perc[0],axis=(0,1)),np.nanpercentile(var,perc[1],axis=(0,1)),color='tab:green',alpha=alpha)#,label=perc)

    
# sea level rise rate
dax = ax[1]

#Rignot
time = np.arange(1950,2018)
X = np.array(slr_rig_df.Antarctica)
XX = 10*np.diff(X)
dax.plot(time[29:],XX[29:],c='tab:olive',lw=4,zorder=10,label='Observations')

#Plot no feedback
var = 1000*(fe.slr[0,1,:,:,1:].values-fe.slr[0,1,:,:,:-1].values)
dax.plot(fe.time[1:],np.nanmedian(var,axis=(0,1)),c='.5',lw=2,label='No feedback')
dax.fill_between(fe.time[1:],np.nanpercentile(var,perc[0],axis=(0,1)),np.nanpercentile(var,perc[1],axis=(0,1)),color='.5',alpha=alpha)#,label=perc)

#Plot with feedback
var = 1000*(fe.slr[1,1,:,:,1:].values-fe.slr[1,1,:,:,:-1].values)
dax.plot(fe.time[1:],np.nanmedian(var,axis=(0,1)),c='tab:green',lw=2,label='With feedback')
dax.fill_between(fe.time[1:],np.nanpercentile(var,perc[0],axis=(0,1)),np.nanpercentile(var,perc[1],axis=(0,1)),color='tab:green',alpha=alpha)#,label=perc)

ax[0].legend(loc='upper left')

ax[0].set_ylabel('Cumulative sea-level rise relative to 1979 [cm]')
ax[1].set_ylabel('Sea level rise rate [mm/yr]')
ax[0].set_title('a)',loc='left')
ax[1].set_title('b)',loc='left')

fig.supxlabel('Time [years]')

ax[0].set_xlim([1950,2017])

plt.savefig(f'../draftfigs/historical_{option}.png',dpi=450,facecolor='w',transparent=False)
plt.show()